In [263]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [264]:
data=pd.read_csv("merged_len3.csv",encoding="shift-jis",index_col=0)
# drop never treated group
data=data.dropna(subset=["Approved"])
data

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,Approved,KEGG,Ingredient,variable
3,2008,114,解熱鎮痛消炎剤,113733431,32062003,0,16419098,1829448,0,1985.0,D00141,Indometacin,id1
4,2008,114,解熱鎮痛消炎剤,113733431,32062003,0,16419098,1829448,0,1985.0,D00126,Ibuprofen,id1
5,2008,114,解熱鎮痛消炎剤,113733431,32062003,0,16419098,1829448,0,1994.0,D00127,Piroxicam,id1
6,2008,114,解熱鎮痛消炎剤,113733431,32062003,0,16419098,1829448,0,1994.0,D00132,Ketoprofen,id1
7,2008,114,解熱鎮痛消炎剤,113733431,32062003,0,16419098,1829448,0,2002.0,D01578,Pranoprofen,id1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3356,2021,625,抗ウイルス剤,215855691,364231,364231,185416723,227307,227307,2007.0,D00222,Aciclovir,id2
3357,2021,625,抗ウイルス剤,215855691,364231,364231,185416723,227307,227307,2009.0,D06298,Vidarabine,id1
3358,2021,629,その他の化学療法剤,56083192,0,0,12140098,0,0,2002.0,D02219,Terbinafine hydrochloride,id2
3359,2021,629,その他の化学療法剤,56083192,0,0,12140098,0,0,2011.0,D00282,Clotrimazole,id3


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [265]:
data["elasped"]=data["year"]-data["Approved"]
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
colnames=dummies.columns.to_list()
# count ver.
# aggregated_data = pd.concat([data.groupby(['year', 'id']).first(), dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index()
# dummy ver.
aggregated_df = pd.concat([
    data.groupby(['year', 'id']).first(), 
    dummies.groupby([data['year'], data['id']]).apply(lambda x: (x[colnames] > 0).any().astype(int))
], axis=1).reset_index()
aggregated_df

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,Approved,...,elasped_29,elasped_30,elasped_31,elasped_32,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38
0,2008,114,解熱鎮痛消炎剤,113733431,32062003,0,16419098,1829448,0,1985.0,...,0,0,0,0,0,0,0,0,0,0
1,2008,121,局所麻酔剤,12649530,0,0,1676687,0,0,1995.0,...,0,0,0,0,0,0,0,0,0,0
2,2008,123,自律神経剤,5180925,0,0,338320,0,0,2006.0,...,0,0,0,0,0,0,0,0,0,0
3,2008,124,鎮けい剤,19908763,297779,0,1360015,128,0,1987.0,...,0,0,0,0,0,0,0,0,0,0
4,2008,131,眼科用剤,200817924,43657879,0,27228595,6132288,0,1997.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,2021,441,抗ヒスタミン剤,2350576,1798389,470949,724732,588014,209582,1990.0,...,0,0,1,0,0,0,0,0,0,0
486,2021,449,その他のアレルギー用薬,226049764,7060159,6962008,51409796,4606525,4528553,1997.0,...,0,0,0,0,0,0,0,0,0,0
487,2021,625,抗ウイルス剤,215855691,364231,364231,185416723,227307,227307,2007.0,...,0,0,0,0,0,0,0,0,0,0
488,2021,629,その他の化学療法剤,56083192,0,0,12140098,0,0,2002.0,...,0,0,0,0,0,0,0,0,0,0


In [266]:
colnames

['elasped_m14',
 'elasped_m13',
 'elasped_m12',
 'elasped_m11',
 'elasped_m10',
 'elasped_m9',
 'elasped_m8',
 'elasped_m7',
 'elasped_m6',
 'elasped_m5',
 'elasped_m4',
 'elasped_m3',
 'elasped_m2',
 'elasped_m1',
 'elasped_0',
 'elasped_1',
 'elasped_2',
 'elasped_3',
 'elasped_4',
 'elasped_5',
 'elasped_6',
 'elasped_7',
 'elasped_8',
 'elasped_9',
 'elasped_10',
 'elasped_11',
 'elasped_12',
 'elasped_13',
 'elasped_14',
 'elasped_15',
 'elasped_16',
 'elasped_17',
 'elasped_18',
 'elasped_19',
 'elasped_20',
 'elasped_21',
 'elasped_22',
 'elasped_23',
 'elasped_24',
 'elasped_25',
 'elasped_26',
 'elasped_27',
 'elasped_28',
 'elasped_29',
 'elasped_30',
 'elasped_31',
 'elasped_32',
 'elasped_33',
 'elasped_34',
 'elasped_35',
 'elasped_36',
 'elasped_37',
 'elasped_38']

In [267]:
dusage=pd.read_csv("./generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [268]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,Approved,...,elasped_31,elasped_32,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38,Year,Percentage
0,2008,114,解熱鎮痛消炎剤,113733431,32062003,0,16419098,1829448,0,1985.0,...,0,0,0,0,0,0,0,0,2008,35.35
1,2008,121,局所麻酔剤,12649530,0,0,1676687,0,0,1995.0,...,0,0,0,0,0,0,0,0,2008,35.35
2,2008,123,自律神経剤,5180925,0,0,338320,0,0,2006.0,...,0,0,0,0,0,0,0,0,2008,35.35
3,2008,124,鎮けい剤,19908763,297779,0,1360015,128,0,1987.0,...,0,0,0,0,0,0,0,0,2008,35.35
4,2008,131,眼科用剤,200817924,43657879,0,27228595,6132288,0,1997.0,...,0,0,0,0,0,0,0,0,2008,35.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,2021,441,抗ヒスタミン剤,2350576,1798389,470949,724732,588014,209582,1990.0,...,1,0,0,0,0,0,0,0,2021,79.00
486,2021,449,その他のアレルギー用薬,226049764,7060159,6962008,51409796,4606525,4528553,1997.0,...,0,0,0,0,0,0,0,0,2021,79.00
487,2021,625,抗ウイルス剤,215855691,364231,364231,185416723,227307,227307,2007.0,...,0,0,0,0,0,0,0,0,2021,79.00
488,2021,629,その他の化学療法剤,56083192,0,0,12140098,0,0,2002.0,...,0,0,0,0,0,0,0,0,2021,79.00


In [269]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic%"},inplace=True)
df_with_dummies

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,Approved,...,id_321,id_332,id_339,id_391,id_399,id_441,id_449,id_625,id_629,id_799
0,2008,114,解熱鎮痛消炎剤,113733431,32062003,0,16419098,1829448,0,1985.0,...,0,0,0,0,0,0,0,0,0,0
1,2008,121,局所麻酔剤,12649530,0,0,1676687,0,0,1995.0,...,0,0,0,0,0,0,0,0,0,0
2,2008,123,自律神経剤,5180925,0,0,338320,0,0,2006.0,...,0,0,0,0,0,0,0,0,0,0
3,2008,124,鎮けい剤,19908763,297779,0,1360015,128,0,1987.0,...,0,0,0,0,0,0,0,0,0,0
4,2008,131,眼科用剤,200817924,43657879,0,27228595,6132288,0,1997.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,2021,441,抗ヒスタミン剤,2350576,1798389,470949,724732,588014,209582,1990.0,...,0,0,0,0,0,1,0,0,0,0
486,2021,449,その他のアレルギー用薬,226049764,7060159,6962008,51409796,4606525,4528553,1997.0,...,0,0,0,0,0,0,1,0,0,0
487,2021,625,抗ウイルス剤,215855691,364231,364231,185416723,227307,227307,2007.0,...,0,0,0,0,0,0,0,1,0,0
488,2021,629,その他の化学療法剤,56083192,0,0,12140098,0,0,2002.0,...,0,0,0,0,0,0,0,0,1,0


In [270]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
df_with_dummies.to_csv("len3_drop_never.csv",encoding="shift-jis")